<a href="https://colab.research.google.com/github/abiaggi/artificial-flavor-gpt2/blob/main/Card_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import requests
from tqdm import tqdm

path = "json"
file_name = "cards.json"

if not os.path.exists(path):
    os.makedirs(path)
    
#url = "https://mtgjson.com/api/v5/AtomicCards.json"
url = "https://mtgjson.com/api/v5/AllPrintings.json"
# url = "https://mtgjson.com/api/v5/STA.json"
r = requests.get(url, stream=True)
with open(os.path.join(path, file_name), 'wb') as f:
    f.write(r.content)

In [8]:
CARD_FORMAT = "Name: {0} | Rarity: {1} | Cost: {2} | Type: {3} | Stats: {4} - {5} | Text: {6} | FlavorText: {7} | Loyalty: {8}"
def card2out(card):
  return CARD_FORMAT.format(card.get('name'), card.get('rarity'), card.get('manaCost'), card.get('type'), card.get('power'), card.get('toughness'), card.get('text'), card.get('flavorText'), card.get('loyalty'))

In [9]:
import json
with open(os.path.join(path, file_name), encoding='utf-8') as f:
    all_cards = json.load(f)

card_dict = {}

for set_exp in all_cards['data']:
    set_cards = all_cards['data'][set_exp]['cards'];
    for card in set_cards:
       card_dict[card['name']] = card2out(card)

In [11]:
if not os.path.exists('out'):
    os.makedirs('out')
with open(os.path.join('out', 'cards.txt'), 'w', encoding='utf-8') as f:
    for card_name in card_dict:
        f.write(' '.join(card_dict[card_name].split()) + '\n')

In [12]:
model = "117M"

subdir = os.path.join('models', model)
if not os.path.exists(subdir):
    os.makedirs(subdir)
subdir = subdir.replace('\\','/') # needed for Windows

for filename in ['checkpoint','encoder.json','hparams.json','model.ckpt.data-00000-of-00001', 'model.ckpt.index', 'model.ckpt.meta', 'vocab.bpe']:

    r = requests.get("https://openaipublic.blob.core.windows.net/gpt-2/" + subdir + "/" + filename, stream=True)

    with open(os.path.join(subdir, filename), 'wb') as f:
        file_size = int(r.headers["content-length"])
        chunk_size = 1000
        with tqdm(ncols=100, desc="Fetching " + filename, total=file_size, unit_scale=True) as pbar:
            # 1k for chunk_size, since Ethernet packet size is around 1500 bytes
            for chunk in r.iter_content(chunk_size=chunk_size):
                f.write(chunk)
                pbar.update(chunk_size)

Fetching checkpoint: 1.00kit [00:00, 427kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 2.57Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 838kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:18, 26.6Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 2.38Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 1.32Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 1.29Mit/s]                                                       


In [13]:
pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 38kB/s 
     |████████████████████████████████| 3.8MB 43.1MB/s 
     |████████████████████████████████| 512kB 55.2MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=ebf24e52a0e416ca26c072cd8071ffd49ca4a8c6aee597b823b6ea0e3874bc2f
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

In [14]:
pip install gpt2-client

  Created wheel for gpt2-client: filename=gpt2_client-2.1.5-cp37-none-any.whl size=16434 sha256=5922ff19b657a6d1f90bc10fa8e2c54ecfd6d294d7e759c4058a54ecdd81571b
  Stored in directory: /root/.cache/pip/wheels/47/03/99/deadcb428f1b3c591edb8d6a2ac0136bb4326a266364b72153
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.2-cp37-none-any.whl size=23621 sha256=a8f45ba5a69355fa1636ab1d0f5fd8f5ca75e389ce0cb789e37a339542deae1a
  Stored in directory: /root/.cache/pip/wheels/68/1d/15/c87a4302a6c7273ce1b4f282bec3c6877fb2f521535d87d30f
Successfully built gpt2-client gpt-2-simple


In [15]:
from gpt2_client import GPT2Client

gpt2 = GPT2Client('117M')  # This could also be `345M`, `774M`, or `1558M`.
gpt2.load_model(force_download=False)  # Use cached versions if available.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Loading checkpoint                             File already exists 
Loading encoder.json                           File already exists 
Loading hparams.json                           File already exists 
Loading model.ckpt.data-00000-of-00001         File already exists 
Loading model.ckpt.index                       File already exists 
Loading model.ckpt.meta                        File already exists 
Loading vocab.bpe                              File already exists 


In [ ]:
corpus = "out/flavor.txt"
custom = gpt2.finetune(corpus, return_text=True)

Loading checkpoint models/117M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:23<00:00, 23.70s/it]


dataset has 5612154 tokens
Training...
[1 | 7.27] loss=1.88 avg=1.88
[2 | 8.49] loss=1.68 avg=1.78
[3 | 9.72] loss=1.79 avg=1.78
[4 | 10.94] loss=1.52 avg=1.72
[5 | 12.17] loss=1.50 avg=1.67
[6 | 13.40] loss=1.57 avg=1.66
[7 | 14.63] loss=1.54 avg=1.64
[8 | 15.85] loss=1.61 avg=1.64
[9 | 17.08] loss=1.47 avg=1.62
[10 | 18.30] loss=1.58 avg=1.61
[11 | 19.53] loss=1.60 avg=1.61
[12 | 20.76] loss=1.53 avg=1.61
[13 | 21.99] loss=1.53 avg=1.60
[14 | 23.21] loss=1.43 avg=1.59
[15 | 24.44] loss=1.50 avg=1.58
[16 | 25.66] loss=1.50 avg=1.57
[17 | 26.90] loss=1.36 avg=1.56
[18 | 28.12] loss=1.52 avg=1.56
[19 | 29.35] loss=1.63 avg=1.56
[20 | 30.58] loss=1.48 avg=1.56
[21 | 31.80] loss=1.51 avg=1.56
[22 | 33.02] loss=1.45 avg=1.55
[23 | 34.25] loss=1.42 avg=1.54
[24 | 35.48] loss=1.39 avg=1.54
[25 | 36.70] loss=1.52 avg=1.54
[26 | 37.93] loss=1.38 avg=1.53
[27 | 39.15] loss=1.44 avg=1.53
[28 | 40.38] loss=1.47 avg=1.52
[29 | 41.60] loss=1.48 avg=1.52
[30 | 42.83] loss=1.36 avg=1.52
[31 | 44.05] 